# Tutoriel de Cartographie 3D du Terrain d'Émulation Planétaire Canadien
***

**Tutoriel :** Ce tutoriel fournit des conseils pour créer une carte 3D du Terrain d'Émulation de Mars de l'Agence spatiale canadienne via Python. Le MET de l'ASC est une installation d'essai extérieure avec des dimensions d'espace de travail de 120m x 60m. Le terrain se compose de roches dispersées sur le sable, ainsi que de quelques grandes caractéristiques de crête, cratère et affleurement.
**Mission et Instrument :** MobileRobots Pioneer P2AT modifié
**Cible :** Terrain d'Émulation Planétaire Canadien
**Exigences du système :** Accès à Internet
**Niveau du tutoriel :** Intermédiaire
**À propos :** L'ensemble de données se compose de 102 balayages laser obtenus à l'aide d'un MobileRobots Pioneer P2AT modifié au Terrain d'Émulation de Mars (MET) de l'Agence spatiale canadienne (ASC), situé près de Montréal, Québec, Canada. Cet ensemble de données a été collecté à des fins de cartographie en octobre 2010. Pour plus d'informations sur l'ensemble de données, visitez : [Ensemble de données de cartographie 3D d'émulation planétaire canadienne - Portail de données ouvertes de l'Agence spatiale canadienne](https://donnees-data.asc-csa.gc.ca/en/dataset/65376529-3z6l-6u7e-732sbzy824wa25) et la [documentation du laboratoire de robotique spatiale autonome de l'ensemble de données de cartographie 3D d'émulation planétaire canadienne](http://asrl.utias.utoronto.ca/datasets/3dmap/).<br>

In [ ]:
#Import the necessary libraries
import os
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

In [ ]:
#Download the dataset
pcd_path = r'C:\Users\rsheikholmolouki\Desktop\p2at_met'

In [ ]:
file_list = []
transform_list = []
#Traverse the directory to collect file and trasnform lists
for root, dirs, files in os.walk(pcd_path):
    for file in files:
        if file.endswith ('.xyz'): #.xyz file refers to Cartesian Laser Data
            file_list.append(os.path.join(root,file))
            
        if file.endswith ('.gt'): #.gt file refers to Ground trurth sensor poses
            transform_list.append(os.path.join(root,file))

In [ ]:
#Loading point cloud data and creating a point cloud object
pcd_data = []
for file in file_list:
    try:
        data= np.loadtxt(file,delimiter = ' ', dtype = float)
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(data[:,:3])
        pcd_data.append(pcd)
        del pcd
    except ValueError as e:
        print (f'Error loading file {file}:{e}')

In [ ]:
#Opening and reading the contents of the tranform file
f = open(transform_list[1], "r")
print(f.read())

In [ ]:
#Accessing tranform list elements
print(transform_list[0],transform_list[1] )

In [ ]:
#Combine the point cloud data with corresponding transforms
pcd_combined = o3d.geometry.PointCloud()

for i in range(len(transform_list)): 
    T = np.loadtxt(transform_list[i])
    pcd_combined += pcd_data[i].transform(T)

In [ ]:
print(pcd_combined)
#Visualize the point cloud after transformation
o3d.visualization.draw_geometries([pcd_combined])

![Mars%20Yard.png](attachment:Mars%20Yard.png)

In [ ]:
downsampled_pcd = pcd_combined.voxel_down_sample(voxel_size= 0.02)
print(downsampled_pcd)
#Visualize the point cloud after transformation
o3d.visualization.draw_geometries([downsampled_pcd])

![down.png](attachment:down.png)

In [ ]:
#statistical removal of outliers, the lower the std_ratio the more aggressive it is
cl_pcd, ind = downsampled_pcd.remove_statistical_outlier(nb_neighbors=40, std_ratio= 3.75)
o3d.visualization.draw_geometries([cl_pcd])

![horizontal.png](attachment:horizontal.png)

![outlier.png](attachment:outlier.png)

### Recadrage Manuel du Mars Yard :
Pour sélectionner manuellement une partie du Mars Yard, les commandes suivantes peuvent être utilisées. Les points seront sauvegardés lorsque la fenêtre se ferme. Dans ce tutoriel, les limites autour du Mars Yard sont recadrées. <br>

**Commandes** :
1) Utilisez x, y, z pour s'orienter dans l'axe<br>
2) f permet une vue libre <br>
3) 1, 2, 3, 4 couleurs utilisant l'éclairage phong, coordonnées x, coordonnées y, coordonnées z (z est le meilleur dans ce cas pour l'éclairage topographique) <br>
4) Shift + 1, 2, 3, 4, 5 sont différentes options de couleur <br>
5) Cliquez sur k pour verrouiller l'édition<br>
6) Maintenez le clic gauche pour dessiner un rectangle <br>
7) Maintenez shift + clic gauche pour dessiner un polygone<br>
8) Cliquez sur c pour recadrer les points<br>

In [ ]:
def pcd_crop_geometry(pcd):
    print("Manual Cropping of Mars Yard")
    vis = o3d.visualization.VisualizerWithEditing()
    vis.create_window()
    vis.add_geometry(pcd)
    vis.run()  
    vis.destroy_window()
    return vis.get_cropped_geometry()
    
cropped_pcd = pcd_crop_geometry(cl_pcd)

![manual.png](attachment:manual.png)

In [ ]:
plane_model, plane_points = cropped_pcd.segment_plane(distance_threshold=0.5,
                                         ransac_n=5,
                                         num_iterations=1000)

inlier_cloud = cropped_pcd.select_by_index(plane_points)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = cropped_pcd.select_by_index(plane_points, invert=True)
outlier_cloud.paint_uniform_color([0, 0, 1.0])

map_bounding_box = inlier_cloud.get_axis_aligned_bounding_box()
map_bounding_box.color = [1.0, 0, 0]

o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud, map_bounding_box])

![manual%20plan.png](attachment:manual%20plan.png)

In [ ]:
border_margin = 0.5
min_bound = map_bounding_box.min_bound - border_margin
max_bound = map_bounding_box.max_bound + border_margin
cropping_box = o3d.geometry.AxisAlignedBoundingBox(min_bound = min_bound, max_bound = max_bound)
cropping_box.color = [0, 1, 0]
o3d.visualization.draw_geometries([cl_pcd, cropping_box])
cropped_pcd = pcd_combined.crop(cropping_box)
o3d.visualization.draw_geometries([cropped_pcd])

![map%20selection%20.png](attachment:map%20selection%20.png)

![Final%20map.png](attachment:Final%20map.png)